## Day 48 Proyect: Cookie Clicker Bot

### Proyecto Día 48 – Cookie Clicker Bot 🍪🤖

Este proyecto consiste en crear un bot que juegue automáticamente al juego [Cookie Clicker](https://orteil.dashnet.org/cookieclicker/), utilizando Selenium.  
El objetivo es maximizar el número de cookies por segundo durante 5 minutos, comprando mejoras y haciendo clic constantemente.

📌 Habilidades aplicadas:
- Automatización con Selenium
- Selección de elementos HTML dinámicos
- Control de tiempo y ciclos
- Extracción y uso de datos en tiempo real


![Cookie Clicker](./CookieClickerBot.png)

*Cookie Clicker*

#### Paso 1 – Configuraciones iniciales

Importamos las librerías necesarias. Así como del diccionario de los productos previamente buscado.

In [1]:
from selenium import webdriver                              # Libreria para controlar el navegador
from selenium.webdriver.common.by import By                 # Libreria para seleccionar elementos
from selenium.webdriver.common.keys import Keys             # Libreria para enviar acciones de teclado
import undetected_chromedriver as uc                        # Libreria para evitar detección de Selenium
import time                                                 # Libreria para manejar el tiempo

In [2]:
# Diccionario para mapear IDs de productos a nombres en español
product_names = {
    "product0": "Cursor",
    "product1": "Abuela",
    "product2": "Granja",
    "product3": "Mina",
    "product4": "Fábrica",
    "product5": "Banco",
    "product6": "Templo",
    "product7": "Torre de hechizos",
    "product8": "Nave espacial",
    "product9": "Laboratorio de alquimia",
    "product10": "Portal",
    "product11": "Máquina del tiempo",
    "product12": "Condensador antimateria",
    "product13": "Prism",
    "product14": "Máquina de Chance",
    "product15": "Fractal engine",
    "product16": "Consola de JavaScript",
    "product17": "Granja idética"
}

#### Paso 2 – Ejecutar el bot durante n minutos 🤖

El bot clickea constantemente, y cada n segundos revisa qué mejora puede comprar.


In [3]:
# Configurar opciones de Chrome
options = uc.ChromeOptions()                                           # Creamos una instancia de ChromeOptions para undetected_chromedriver
options.add_argument("--no-first-run")                                 # Prevenir configuraciones iniciales
options.add_argument("--no-default-browser-check")                     # Prevenir chequeo de navegador por defecto
options.add_argument("--disable-blink-features=AutomationControlled")  # Evitar detección de Selenium
options.add_argument("--disable-infobars")                             # Desactivar la barra de información
options.add_argument("--start-maximized")                            # Opcional: abrir maximizado

# Inicializar navegador con undetected-chromedriver
driver = uc.Chrome(options=options)

# Navegar a la página del juego
driver.get("https://orteil.dashnet.org/cookieclicker/")

# Esperar a que el juego cargue completamente antes de seleccionar idioma
time.sleep(7)

# Seleccionar idioma español
boton_idioma = driver.find_element(By.ID, "langSelect-ES")
boton_idioma.click()

# Esperar a que se cargue el juego después de seleccionar el idioma
time.sleep(3)

# ----------------------------------------------------------------------------
#-----------------------------------------------------------------------------
# Elemento principal: el botón de la cookie
cookie = driver.find_element(By.ID, "bigCookie")

# Elementos de la tienda
items = driver.find_elements(By.CSS_SELECTOR, "#store div")
item_ids = [item.get_attribute("id") for item in items if item.get_attribute("id")]

# Imprimir la cantidad de elementos en la tienda
print(f"Encontrados {len(item_ids)} elementos de tienda🛒")


# ----------------------------------------------------------------------------
#-----------------------------------------------------------------------------

# Generar temporizadores
timeout = time.time() + 5                           # Duración de clicks antes de compra: n segundos desde ahora
end_time = time.time() + 60 * 2                     # Duración de juego: n minutos desde ahora 

# Generar un bucle para hacer clic en la cookie mientras el temporizador esté activo
while time.time() < end_time:
    # Verificar que el elemento es clickeable
    if not cookie.is_displayed():
        print("Esperando a que la cookie sea visible...")
        time.sleep(2)
        cookie = driver.find_element(By.ID, "bigCookie")
    
    # Hacer clic en la cookie    
    cookie.click()
    #driver.execute_script("arguments[0].click();", cookie)     # Hacer clic en el producto usando JavaScript para evitar problemas de |   cli
   
    # Si el tiempo de espera ha pasado, comprar mejoras
    if time.time() > timeout:
        cookie_display = driver.find_element(By.ID, "cookies").text             # Obtener el texto del contador de galletas
        cookie_count = int(cookie_display.split()[0].replace(",", ""))          # Convertir el texto a un número entero

        # Lista para almacenar los productos con su información
        product_elements = []
        
        # Buscar productos disponibles en la tienda y se agregan a la lista
        for i in range(0, 18):                                                  # 18 productos posibles
            try:
                product = driver.find_element(By.ID, f"product{i}")                 # Obtener el elemento del producto
                real_name = product_names.get(f"product{i}", f"product{i}")         # Obtener el nombre real del producto (del diccionario)
                product_elements.append((f"product{i}", product, real_name))        # Agregar el producto a la lista
            except:
                pass
                
        # Diccionarios para productos disponibles y sus nombres
        available_products = {}
        product_name_map = {}  
            
        # Evaluar cada producto
        for prod_id, element, prod_name in product_elements:
            if "disabled" not in element.get_attribute("class"):                    # Verificar si el producto está habilitado
                price_elem = element.find_element(By.CLASS_NAME, "price")           # Obtener el elemento del precio
                if price_elem.text:                                                 # Verificar si el precio no está vacío
                    price = int(price_elem.text.replace(",", ""))                   # Convertir el precio a un número entero
                    available_products[prod_id] = price                             # Agregar el precio del producto al diccionario
                    product_name_map[prod_id] = prod_name                           # Agregar el nombre real al diccionario         
        
        # Filtrar productos que podemos comprar
        if available_products and cookie_count > 0:
            affordable_products = {prod: price for prod, price in available_products.items() 
                                    if price <= cookie_count}      
            print(f"🔎Productos que podemos comprar: {len(affordable_products)}")    

            # Comprar el producto más caro que podamos permitirnos
            if affordable_products:
                to_buy_id = max(affordable_products, key=affordable_products.get)    # Encontrar el producto más caro y obtener su indice
                to_buy_name = product_name_map[to_buy_id]                            # Obtener el nombre real del producto mas caro
                to_buy_price = affordable_products[to_buy_id]                        # Obtener el precio del producto mas caro
                
                print(f"🍪 Contador actual: {cookie_count}")                      # Imprimir el contador de galletas actual 
                print(f"🛒 Comprando: {to_buy_name} por {to_buy_price} galletas")   # Imprimir el ticket de compra
                
                # Hacer clic en el producto para comprarlo
                product_element = driver.find_element(By.ID, to_buy_id)             # Obtener el elemento del producto a comprar
                driver.execute_script("arguments[0].scrollIntoView(true);", product_element)  # Desplazar la vista hacia el producto
                driver.execute_script("arguments[0].click();", product_element)     # Hacer clic en el producto usando JavaScript para evitar problemas de clic
                #product_element.click()                                             # Hacer clic en el producto para comprarlo

                # Confirmar la compra
                print(f"✅ Compra de {to_buy_name} realizada\n")
                
                # Actualizar el timeout para dar tiempo a que se actualice la interfaz
                timeout = time.time() + 5
            else:
                print("❌ No hay productos que se puedan comprar en este momento\n")
                timeout = time.time() + 5
        elif not available_products:
            print("❌ No se encontraron productos disponibles\n")
            timeout = time.time() + 5

# Esperar a que se actualicen las estadísticas
time.sleep(2) 

# Imprimir estadísticas finales
cps_stats = driver.find_element(By.ID, "cookiesPerSecond").text
print(f"\n=== Estadísticas Finales ===\nGalletas por segundo: {cps_stats}")

# Opcional: Si quieres cerrar el navegador, descomenta la siguiente línea:
# driver.quit()

Encontrados 113 elementos de tienda🛒
❌ No se encontraron productos disponibles

❌ No se encontraron productos disponibles

❌ No se encontraron productos disponibles

❌ No se encontraron productos disponibles

❌ No se encontraron productos disponibles

🔎Productos que podemos comprar: 1
🍪 Contador actual: 51
🛒 Comprando: Cursor por 15 galletas
✅ Compra de Cursor realizada

🔎Productos que podemos comprar: 1
🍪 Contador actual: 71
🛒 Comprando: Cursor por 18 galletas
✅ Compra de Cursor realizada

🔎Productos que podemos comprar: 1
🍪 Contador actual: 71
🛒 Comprando: Cursor por 20 galletas
✅ Compra de Cursor realizada

🔎Productos que podemos comprar: 1
🍪 Contador actual: 71
🛒 Comprando: Cursor por 23 galletas
✅ Compra de Cursor realizada

🔎Productos que podemos comprar: 2
🍪 Contador actual: 155
🛒 Comprando: Abuela por 100 galletas
✅ Compra de Abuela realizada

🔎Productos que podemos comprar: 2
🍪 Contador actual: 155
🛒 Comprando: Abuela por 115 galletas
✅ Compra de Abuela realizada

🔎Productos q

#### 🧠 Conclusión

Este proyecto fue una excelente forma de consolidar conocimientos avanzados:

- 🖱️ Interacción automática con interfaces dinámicas (Selenium)
- 🔁 Automatización por ciclos de tiempo
- 📊 Extracción de datos desde la interfaz gráfica
- 🧠 Toma de decisiones automatizada (comprar mejoras según cookies disponibles)